# Job Schduling


In [102]:
import pandas as pd
import heapq as q
from collections import deque

DEBUG = True

jobs_list = {}
deallocation_list = {}
for day in range(31):
    jobs_list[day] = {}
    deallocation_list[day] = {}
    for time in range(24):
        jobs_list[day][time] = []
        deallocation_list[day][time] = []

jobs = []
with open('./JobArrival.txt','r') as fp:
    for line in fp:
#         line = fp.readline()
        data = []
        if len(line.strip()) > 10:
            for word in line.strip().split(' '):
                if word.isdigit():
                    data.append(int(word))
            data.append(data[-1]*data[-2]*data[-3])
            jobs.append(data)

for job in jobs:
    row = {}
    row['ID'] = job[0]
    row['Memory'] = job[3]
    row['CPU'] = job[4]
    row['ExecTime'] = job[5]
    row['GV'] = job[6]
    jobs_list[job[1]][job[2]].append(row)

del jobs, data, line, word
    
class WorkerNode:

    def __init__(self):
        self.available_ram = 64
        self.available_cpu = 24
        self.active_list = []

    def allocate(self, day, hour, ram, cpu):
        self.available_ram -= ram
        self.available_cpu -= cpu
        deallocation_list[day][hour].append([self, ram, cpu])

    def in_use_resource(self):
        return 24 - self.available_cpu, 64 - self.available_ram

def job_queue(strategy, queue, day, time):
    
    new_queue = jobs_list[day][time]
    if strategy == "FCFS":
        new_queue = sorted(queue + new_queue, key=lambda x: x['ID'])
    elif strategy == "SJFQ":
        new_queue = sorted(queue + new_queue, key=lambda x: (x['GV'], x['ID']))
    else:
        new_queue = sorted(queue + new_queue, key=lambda x: (x['ExecTime'], x['ID']))
    return new_queue

def allocate_job(job, allocate_strategy, day, hour):
#     print(job, allocate_strategy, day, hour)
    if allocate_strategy == "FF":
        for node in range(128):
            if worker_node[node].available_ram >= job['Memory'] and worker_node[node].available_cpu >= job['CPU']:
                cp_time = calculate_completion_time(day, hour, job['ExecTime'])
                worker_node[node].allocate(cp_time[0], cp_time[1], job['Memory'], job['CPU'])
                return True
    elif allocate_strategy == "BF":
        best_node = -1
        for node in range(128):
            if worker_node[node].available_ram == job['Memory'] and worker_node[node].available_cpu == job['CPU']:
                best_node = node
                break
            elif worker_node[node].available_ram > job['Memory'] and worker_node[node].available_cpu == job['CPU']:
                if worker_node[node].available_ram < worker_node[best_node].available_ram:
                    best_node = node

            elif worker_node[node].available_ram == job['Memory'] and worker_node[node].available_cpu > job['CPU']:
                if worker_node[node].available_cpu < worker_node[best_node].available_cpu:
                    best_node = node

            elif worker_node[node].available_ram > job['Memory'] and worker_node[node].available_cpu > job['CPU']:
                if best_node == -1:
                    best_node = node
                elif worker_node[node].available_ram < worker_node[best_node].available_ram or worker_node[node].available_cpu < worker_node[best_node].available_cpu:
                    best_node = node

        if best_node != -1:
            cp_time = calculate_completion_time(day, hour, job['ExecTime'])
            worker_node[best_node].allocate(cp_time[0], cp_time[1], job['Memory'], job['CPU'])
            return True
        else:
            return False

    else:
        worst_fit = -1
        for node in range(128):
            if worker_node[node].available_ram >= job['Memory'] and worker_node[node].available_cpu >= job['CPU']:
                if worst_fit == -1:
                    worst_fit = node
                elif worker_node[node].available_ram > worker_node[worst_fit].available_ram and worker_node[node].available_cpu > worker_node[worst_fit].available_cpu:
                    worst_fit = node
                elif worker_node[node].available_cpu > worker_node[worst_fit].available_cpu:
                    worst_fit = node
        if worst_fit != -1:
            cp_time = calculate_completion_time(day, hour, job['ExecTime'])
            worker_node[worst_fit].allocate(cp_time[0], cp_time[1], job['Memory'], job['CPU'])
            return True
        else:
            return False
    return False

def calculate_completion_time(day, hour, exe):
    if hour + exe >= 24:
        day += 1
        hour = (hour + exe) % 24;
    else:
        hour += exe
    return day, hour

def return_average(cpu_usage, ram_usage):
    average_cpu = []
    average_ram = []
    len_of_data = len(cpu_usage) // 24
    for each_day in range(len_of_data):
        average_cpu.append(sum(cpu_usage[each_day * 24: each_day * 24 + 24]) / 24)
        average_ram.append(sum(ram_usage[each_day * 24: each_day * 24 + 24]) / 24)
    return average_cpu, average_ram

def print_cpu_usage(day, hour):
    cpu = 0
    ram = 0
    for node in range(128):
        t = worker_node[node].in_use_resource()
        cpu += t[0]
        ram += t[1]
    return cpu, ram

queue_strategy = 'FCFS'
allocate_strategy = 'FF'

worker_node = []
for i in range(128):
    worker_node.append(WorkerNode())
cpu_usage = []
ram_usage = []

queue = []

for day in range(31):
    for hour in range(24):

        # Perform de-allocation from worker node
        for node in deallocation_list[day][hour]:
            node[0].available_ram += node[1]
            node[0].available_cpu += node[2]

        # Extract fcfs fashion and add to the remaining job list
        job_list = job_queue(queue_strategy, queue, day, hour)

        # check possibillity of allocation for each job
        # if can't allocate then add the remaining in queue

        itr_job_list = deque(job_list)
        del job_list
        queue = []
        
        while len(itr_job_list)>0:
            job = itr_job_list.popleft()
            if not allocate_job(job, allocate_strategy, day, hour):
                queue.append(job)

        # calculate the cpu and ram usage from each worker node
        temp = print_cpu_usage(day, hour)
        cpu_usage.append(temp[0])
        ram_usage.append(temp[1])
        
return_average(cpu_usage, ram_usage)

([1547.4166666666667,
  2033.625,
  1573.7083333333333,
  2402.9583333333335,
  1704.375,
  2010.7916666666667,
  2124.2916666666665,
  2063.4166666666665,
  1988.25,
  2028.5416666666667,
  1713.1666666666667,
  1716.0416666666667,
  1860.4583333333333,
  1952.4166666666667,
  1537.125,
  1808.7083333333333,
  2073.6666666666665,
  1792.75,
  1675.5,
  1993.5,
  1875.875,
  1928.4166666666667,
  2116.75,
  1924.7083333333333,
  1653.5,
  1878.4583333333333,
  2083.9583333333335,
  1686.9583333333333,
  2063.9166666666665,
  2410.6666666666665,
  396.0416666666667],
 [2957.2083333333335,
  3839.4166666666665,
  3023.125,
  4591.666666666667,
  3079.75,
  3795.5416666666665,
  3944.75,
  3865.75,
  3759.75,
  3664.8333333333335,
  3335.4583333333335,
  3142.7916666666665,
  3589.7916666666665,
  3787.375,
  2919.8333333333335,
  3576.4583333333335,
  4024.4166666666665,
  3439.2083333333335,
  3170.25,
  3765.0416666666665,
  3648.5416666666665,
  3809.875,
  4116.041666666667,
  3698.5

In [84]:
import pandas as pd
import heapq as q
from collections import deque

DEBUG = True

jobs = []
with open('./JobArrival.txt','r') as fp:
    for line in fp:
#         line = fp.readline()
        data = []
        if len(line.strip()) > 10:
            for word in line.strip().split(' '):
                if word.isdigit():
                    data.append(int(word))
            data.append(data[-1]*data[-2]*data[-3])
            jobs.append(data)

jobs = pd.DataFrame(jobs)
jobs.columns = ['ID', 'DAY', 'Time', 'Memory', 'CPU', 'ExecTime', 'GV']

class WorkerNode:

    def __init__(self):
        self.available_ram = 64
        self.available_cpu = 24
        self.active_list = []

    def allocate(self, day, hour, ram, cpu):
        self.available_ram -= ram
        self.available_cpu -= cpu
        self.active_list.append((day, hour, ram, cpu))

    def de_allocate(self, day, hour):
        itr_list = self.active_list.copy()
        for active in itr_list:
            if day == active[0] and hour == active[1]:
                self.available_ram += active[2]
                self.available_cpu += active[3]
                self.active_list.remove(active)

    def in_use_resource(self):
        return 24 - self.available_cpu, 64 - self.available_ram

def job_queue(strategy, queue, day, time):
    if strategy == "FCFS":
        new_queue = jobs[(jobs.DAY == day) & (jobs.Time == time)]
        new_queue = pd.concat([queue, new_queue], ignore_index=True)
#         new_queue = new_queue.sort_values(['ID'], ascending=True)
        new_queue.reset_index(drop=True, inplace=True)
        return new_queue.to_dict('records')
    elif strategy == "SJFQ":
        new_queue = jobs[(jobs.DAY == day) & (jobs.Time == time)]
        new_queue = pd.concat([new_queue, queue], ignore_index=True)
        new_queue = new_queue.sort_values(['GV', 'ID'], ascending=[True, True])
        new_queue.reset_index(drop=True, inplace=True)
        return new_queue
    else:
        new_queue = jobs[(jobs.DAY == day) & (jobs.Time == time)]
        new_queue = pd.concat([new_queue, queue], ignore_index=True)
        new_queue = new_queue.sort_values(['ExecTime', 'ID'], ascending=[True, True])
        new_queue.reset_index(drop=True, inplace=True)
        return new_queue

def allocate_job(job, allocate_strategy, day, hour):
#     print(job, allocate_strategy, day, hour)
    if allocate_strategy == "FF":
        for node in range(128):
            if worker_node[node].available_ram >= job['Memory'] and worker_node[node].available_cpu >= job['CPU']:
                cp_time = calculate_completion_time(day, hour, job['ExecTime'])
                worker_node[node].allocate(cp_time[0], cp_time[1], job['Memory'], job['CPU'])
                return True
    elif allocate_strategy == "BF":
        best_node = -1
        for node in range(128):
            if worker_node[node].available_ram == job['Memory'] and worker_node[node].available_cpu == job['CPU']:
                best_node = node
                break
            elif worker_node[node].available_ram > job['Memory'] and worker_node[node].available_cpu == job['CPU']:
                if worker_node[node].available_ram < worker_node[best_node].available_ram:
                    best_node = node

            elif worker_node[node].available_ram == job['Memory'] and worker_node[node].available_cpu > job['CPU']:
                if worker_node[node].available_cpu < worker_node[best_node].available_cpu:
                    best_node = node

            elif worker_node[node].available_ram > job['Memory'] and worker_node[node].available_cpu > job['CPU']:
                if best_node == -1:
                    best_node = node
                elif worker_node[node].available_ram < worker_node[best_node].available_ram or worker_node[node].available_cpu < worker_node[best_node].available_cpu:
                    best_node = node

        if best_node != -1:
            cp_time = calculate_completion_time(day, hour, job['ExecTime'])
            worker_node[best_node].allocate(cp_time[0], cp_time[1], job['Memory'], job['CPU'])
            return True
        else:
            return False

    else:
        worst_fit = -1
        for node in range(128):
            if worker_node[node].available_ram >= job['Memory'] and worker_node[node].available_cpu >= job['CPU']:
                if worst_fit == -1:
                    worst_fit = node
                elif worker_node[node].available_ram > worker_node[worst_fit].available_ram and worker_node[node].available_cpu > worker_node[worst_fit].available_cpu:
                    worst_fit = node
                elif worker_node[node].available_cpu > worker_node[worst_fit].available_cpu:
                    worst_fit = node
        if worst_fit != -1:
            cp_time = calculate_completion_time(day, hour, job['ExecTime'])
            worker_node[worst_fit].allocate(cp_time[0], cp_time[1], job['Memory'], job['CPU'])
            return True
        else:
            return False
    return False

def calculate_completion_time(day, hour, exe):
    if hour + exe >= 24:
        day += 1
        hour = (hour + exe) % 24;
    else:
        hour += exe
    return day, hour

def return_average(cpu_usage, ram_usage):
    average_cpu = []
    average_ram = []
    len_of_data = len(cpu_usage) // 24
    for each_day in range(len_of_data):
        average_cpu.append(sum(cpu_usage[each_day * 24: each_day * 24 + 24]) / 24)
        average_ram.append(sum(ram_usage[each_day * 24: each_day * 24 + 24]) / 24)
    return average_cpu, average_ram

def print_cpu_usage(day, hour):
    cpu = 0
    ram = 0
    for node in range(128):
        t = worker_node[node].in_use_resource()
        cpu += t[0]
        ram += t[1]
    return cpu, ram

queue_strategy = 'FCFS'
allocate_strategy = 'FF'

worker_node = []
for i in range(128):
    worker_node.append(WorkerNode())
cpu_usage = []
ram_usage = []

queue = jobs[0:0].copy()

for day in range(jobs['DAY'].max() + 1):
    for hour in range(jobs['Time'].max() + 1):

        # Perform de-allocation from worker node
        for node in range(128):
            worker_node[node].de_allocate(day, hour)

        # Extract fcfs fashion and add to the remaining job list
        job_list = job_queue(queue_strategy, queue, day, hour)

        # check possibillity of allocation for each job
        # if can't allocate then add the remaining in queue
#         itr_job_list = job_list.copy()
        itr_job_list = job_list.copy()
        for job in itr_job_list:
            if allocate_job(job, allocate_strategy, day, hour):
                job_list.remove(job)

        # calculate the cpu and ram usage from each worker node
        temp = print_cpu_usage(day, hour)
        cpu_usage.append(temp[0])
        ram_usage.append(temp[1])

        # add unallocated jobs
        queue = queue[0:0].copy()
        for job in job_list:
            queue = queue.append(job, ignore_index=True)
        del job_list
        
return_average(cpu_usage, ram_usage)

([1547.4166666666667,
  2033.625,
  1573.7083333333333,
  2402.9583333333335,
  1704.375,
  2010.7916666666667,
  2124.2916666666665,
  2063.4166666666665,
  1988.25,
  2028.5416666666667,
  1713.1666666666667,
  1716.0416666666667,
  1860.4583333333333,
  1952.4166666666667,
  1537.125,
  1808.7083333333333,
  2073.6666666666665,
  1792.75,
  1675.5,
  1993.5,
  1875.875,
  1928.4166666666667,
  2116.75,
  1924.7083333333333,
  1653.5,
  1878.4583333333333,
  2083.9583333333335,
  1686.9583333333333,
  2063.9166666666665,
  2410.6666666666665],
 [2957.2083333333335,
  3839.4166666666665,
  3023.125,
  4591.666666666667,
  3079.75,
  3795.5416666666665,
  3944.75,
  3865.75,
  3759.75,
  3664.8333333333335,
  3335.4583333333335,
  3142.7916666666665,
  3589.7916666666665,
  3787.375,
  2919.8333333333335,
  3576.4583333333335,
  4024.4166666666665,
  3439.2083333333335,
  3170.25,
  3765.0416666666665,
  3648.5416666666665,
  3809.875,
  4116.041666666667,
  3698.5416666666665,
  3042.

In [82]:
"""
created by Suraj Kesharwani
"""
import re
import pandas as pd
import matplotlib.pyplot as plt
import time

t = time.time()

try:
    class WorkerNode:

        def __init__(self):
            self.available_ram = 64
            self.available_cpu = 24
            self.active_list = []  # 0 -> Day; 1 -> Hour; 2-> Occupied RAM; 3-> Occupied CPU

        def allocate(self, day, hour, ram, cpu):
            self.available_ram -= ram
            self.available_cpu -= cpu
            self.active_list.append((day, hour, ram, cpu))

        def de_allocate(self, day, hour):
            itr_list = self.active_list.copy()
            for active in itr_list:
                if day == active[0] and hour == active[1]:
                    self.available_ram += active[2]
                    self.available_cpu += active[3]
                    self.active_list.remove(active)

        def in_use_resource(self):
            return 24 - self.available_cpu, 64 - self.available_ram


    class MasterNode:

        def __init__(self, df):
            worker_node = []
            self.df = df
            self.cpu_usage = []
            self.ram_usage = []

        def return_average(self, cpu_usage, ram_usage):
            average_cpu = []
            average_ram = []
            len_of_data = len(cpu_usage) // 24
            for each_day in range(len_of_data):
                average_cpu.append(sum(cpu_usage[each_day * 24: each_day * 24 + 24]) / 24)
                average_ram.append(sum(ram_usage[each_day * 24: each_day * 24 + 24]) / 24)
            return average_cpu, average_ram

        def print_cpu_usage(self, day, hour):
            cpu = 0
            ram = 0
            for node in range(128):
                t = self.worker_node[node].in_use_resource()
                cpu += t[0]
                ram += t[1]
            return cpu, ram

        def queue_job(self, queue_strategy, queue, day, hour):
            if queue_strategy == "FCFS":
                cur_df = self.df.loc[(self.df['Day'] == day) & (self.df['Hour'] == hour)]
                cur_df = pd.concat([cur_df, queue], ignore_index=True)
                cur_df = cur_df.sort_values(['JobId'], ascending=True)
                job_list = cur_df.to_dict('records')
                return job_list
            elif queue_strategy == "SJFQ":
                cur_df = self.df.loc[(self.df['Day'] == day) & (self.df['Hour'] == hour)]
                # print("Before Sort")
                # print(cur_df.head(10))
                cur_df = pd.concat([cur_df, queue], ignore_index=True)
                cur_df = cur_df.sort_values(['gross_val', 'JobId'], ascending=[True, True])
                # print("After Sort")
                # print(cur_df.head(10))

                job_list = cur_df.to_dict('records')
                return job_list
            else:
                cur_df = self.df.loc[(self.df['Day'] == day) & (self.df['Hour'] == hour)]
                # print("Before Sort")
                # print(cur_df.head(10))
                cur_df = pd.concat([cur_df, queue], ignore_index=True)
                cur_df = cur_df.sort_values(['ExeTime', 'JobId'], ascending=[True, True])
                # print("After Sort")
                # print(cur_df.head(10))
                job_list = cur_df.to_dict('records')
                return job_list

        def find_usage(self, queue_strategy="FCFS", allocate_strategy="FF"):
            self.worker_node = []
            for i in range(128):
                self.worker_node.append(WorkerNode())
            self.cpu_usage = []
            self.ram_usage = []

            queue = self.df[0:0].copy()
            for day in range(max(self.df['Day']) + 1):
                for hour in range(max(self.df['Hour']) + 1):

                    # Perform de-allocation from worker node
                    for node in range(128):
                        self.worker_node[node].de_allocate(day, hour)

                    # Extract fcfs fashion and add to the remaining job list
                    job_list = self.queue_job(queue_strategy, queue, day, hour)

                    # check possibillity of allocation for each job
                    # if can't allocate then add the remaining in queue
                    itr_job_list = job_list.copy()
                    for job in itr_job_list:
                        if self.allocate_job(job, allocate_strategy, day, hour):
                            job_list.remove(job)

                    # calculate the cpu and ram usage from each worker node
                    temp = self.print_cpu_usage(day, hour)
                    self.cpu_usage.append(temp[0])
                    self.ram_usage.append(temp[1])

                    # add unallocated jobs
                    queue = queue[0:0].copy()
                    for job in job_list:
                        queue = queue.append(job, ignore_index=True)

            return self.return_average(self.cpu_usage, self.ram_usage)

        def allocate_job(self, job, allocate_strategy, day, hour):
            if allocate_strategy == "FF":
                for node in range(128):
                    if self.worker_node[node].available_ram >= job['MemReq'] and self.worker_node[node].available_cpu >= job['CPUReg']:
                        cp_time = self.calculate_completion_time(day, hour, job['ExeTime'])
                        self.worker_node[node].allocate(cp_time[0], cp_time[1], job['MemReq'], job['CPUReg'])
                        return True
            elif allocate_strategy == "BF":
                best_node = -1
                for node in range(128):
                    if self.worker_node[node].available_ram == job['MemReq'] and self.worker_node[node].available_cpu == job['CPUReg']:
                        best_node = node
                        break
                    elif self.worker_node[node].available_ram > job['MemReq'] and self.worker_node[node].available_cpu == job['CPUReg']:
                        if self.worker_node[node].available_ram < self.worker_node[best_node].available_ram:
                            best_node = node

                    elif self.worker_node[node].available_ram == job['MemReq'] and self.worker_node[node].available_cpu > job['CPUReg']:
                        if self.worker_node[node].available_cpu < self.worker_node[best_node].available_cpu:
                            best_node = node

                    elif self.worker_node[node].available_ram > job['MemReq'] and self.worker_node[node].available_cpu > job['CPUReg']:
                        if best_node == -1:
                            best_node = node
                        elif self.worker_node[node].available_ram < self.worker_node[best_node].available_ram or self.worker_node[node].available_cpu < self.worker_node[best_node].available_cpu:
                            best_node = node

                if best_node != -1:
                    cp_time = self.calculate_completion_time(day, hour, job['ExeTime'])
                    self.worker_node[best_node].allocate(cp_time[0], cp_time[1], job['MemReq'], job['CPUReg'])
                    return True
                else:
                    return False

            else:
                worst_fit = -1
                for node in range(128):
                    if self.worker_node[node].available_ram >= job['MemReq'] and self.worker_node[node].available_cpu >= job['CPUReg']:
                        if worst_fit == -1:
                            worst_fit = node
                        elif self.worker_node[node].available_ram > self.worker_node[worst_fit].available_ram and self.worker_node[node].available_cpu > self.worker_node[worst_fit].available_cpu:
                            worst_fit = node
                        elif self.worker_node[node].available_cpu > self.worker_node[worst_fit].available_cpu:
                            worst_fit = node
                if worst_fit != -1:
                    cp_time = self.calculate_completion_time(day, hour, job['ExeTime'])
                    self.worker_node[worst_fit].allocate(cp_time[0], cp_time[1], job['MemReq'], job['CPUReg'])
                    return True
                else:
                    return False
            return False

        def calculate_completion_time(self, day, hour, exe):
            if hour + exe >= 24:
                day += 1
                hour = (hour + exe) % 24;
            else:
                hour += exe
            return day, hour


    def process_line(line):
        file_data = re.sub('[\s\s]+', ' ', line).split(' ')
        file_data = {'JobId': int(file_data[file_data.index('JobId:') + 1]),
                     'Day': int(file_data[file_data.index('Day:') + 1]),
                     'Hour': int(file_data[file_data.index('Hour:') + 1]),
                     'MemReq': int(file_data[file_data.index('MemReq:') + 1]),
                     'CPUReg': int(file_data[file_data.index('CPUReg:') + 1]),
                     'ExeTime': int(file_data[file_data.index('ExeTime:') + 1])}
        return file_data

    def give_percentage(cpu_usage, ram_usage):
        c_mul = 100 / (128 * 24)
        r_mul = 100 / (128 * 64)
        cpu_usage = [cpu_usage[i] * c_mul for i in range(len(cpu_usage))]
        ram_usage = [ram_usage[i] * r_mul for i in range(len(ram_usage))]
        return cpu_usage, ram_usage


    with open('JobArrival.txt') as f:
        data = f.readlines()

    df_data = []
    for row in range(len(data)):
        if 'Job' in data[row]:
            df_data.append(process_line(data[row]))

    df = pd.DataFrame(df_data)
    df['gross_val'] = df['ExeTime'] * df['CPUReg'] * df['MemReq']
    ms_node = MasterNode(df)
    day = [i for i in range(1, 31)]

    # 1st Combination
    print("\nFCFS with First Fit")
    fcfs_ff = ms_node.find_usage("FCFS", "FF")
    print(fcfs_ff)

#---------------------------------------------------------------------------------------------------------------------
    # 2nd Combination
#     print("\nFCFS with Best Fit")
#     fcfs_bf = ms_node.find_usage("FCFS", "BF")

#     print("CPU Usage :", fcfs_bf[0], "\nMem Usage :", fcfs_bf[1])

#     fcfs_bf = give_percentage(fcfs_bf[0], fcfs_bf[1])
#     # print("CPU Usage in %:", fcfs_ff[0], "\nMem Usage in %:", fcfs_ff[1])

#     fcfs_bf_fig = plt.figure(2)
#     plt.plot(day, fcfs_bf[0], 'b', label="CPU Usage")
#     plt.plot(day, fcfs_bf[1], 'g', label="Mem Usage")
#     plt.legend()
#     plt.title('FCFS - Best Fit Usage')
# # ---------------------------------------------------------------------------------------------------------------------
#     # 3rd Combination
#     print("\nFCFS with Worst Fit")
#     fcfs_wf = ms_node.find_usage("FCFS", "WF")

#     print("CPU Usage :", fcfs_wf[0], "\nMem Usage :", fcfs_wf[1])

#     fcfs_wf = give_percentage(fcfs_wf[0], fcfs_wf[1])
#     # print("CPU Usage in %:", fcfs_ff[0], "\nMem Usage in %:", fcfs_ff[1])

#     fcfs_wf_fig = plt.figure(3)
#     plt.plot(day, fcfs_wf[0], 'b', label="CPU Usage")
#     plt.plot(day, fcfs_wf[1], 'g', label="Mem Usage")
#     plt.legend()
#     plt.title('FCFS - Worst Fit Usage')
# # ---------------------------------------------------------------------------------------------------------------------
#     # 4th  Combination
#     print("\nShortest Job First with First Fit")
#     sjfq_ff = ms_node.find_usage("SJFQ", "FF")

#     print("CPU Usage :", sjfq_ff[0], "\nMem Usage :", sjfq_ff[1])

#     sjfq_ff = give_percentage(sjfq_ff[0], sjfq_ff[1])
#     # print("CPU Usage in %:", fcfs_ff[0], "\nMem Usage in %:", fcfs_ff[1])

#     sjfq_ff_fig = plt.figure(4)
#     plt.plot(day, sjfq_ff[0], 'b', label="CPU Usage")
#     plt.plot(day, sjfq_ff[1], 'g', label="Mem Usage")
#     plt.legend()
#     plt.title('Shortest Job First - First Fit Usage')
# # ---------------------------------------------------------------------------------------------------------------------
#     # 5th  Combination
#     print("\nShortest Job First with Best Fit")
#     sjfq_bf = ms_node.find_usage("SJFQ", "BF")

#     print("CPU Usage :", sjfq_bf[0], "\nMem Usage :", sjfq_bf[1])

#     sjfq_bf = give_percentage(sjfq_bf[0], sjfq_bf[1])
#     # print("CPU Usage in %:", fcfs_ff[0], "\nMem Usage in %:", fcfs_ff[1])

#     sjfq_bf_fig = plt.figure(5)
#     plt.plot(day, sjfq_bf[0], 'b', label="CPU Usage")
#     plt.plot(day, sjfq_bf[1], 'g', label="Mem Usage")
#     plt.legend()
#     plt.title('Shortest Job First - Best Fit Usage')
# # ---------------------------------------------------------------------------------------------------------------------
#     # 6th  Combination
#     print("\nShortest Job First with Worst Fit")
#     sjfq_wf = ms_node.find_usage("SJFQ", "WF")

#     print("CPU Usage :", sjfq_wf[0], "\nMem Usage :", sjfq_wf[1])

#     sjfq_wf = give_percentage(sjfq_wf[0], sjfq_wf[1])
#     # print("CPU Usage in %:", fcfs_ff[0], "\nMem Usage in %:", fcfs_ff[1])

#     sjfq_wf_fig = plt.figure(6)
#     plt.plot(day, sjfq_wf[0], 'b', label="CPU Usage")
#     plt.plot(day, sjfq_wf[1], 'g', label="Mem Usage")
#     plt.legend()
#     plt.title('Shortest Job First - Worst Fit Usage')
# # ---------------------------------------------------------------------------------------------------------------------
#     # 7th  Combination
#     print("\nShortest Duration Job First with First Fit")
#     sdjfq_ff = ms_node.find_usage("SDJFQ", "FF")

#     print("CPU Usage :", sdjfq_ff[0], "\nMem Usage :", sdjfq_ff[1])

#     sdjfq_ff = give_percentage(sdjfq_ff[0], sdjfq_ff[1])
#     # print("CPU Usage in %:", fcfs_ff[0], "\nMem Usage in %:", fcfs_ff[1])

#     sdjfq_ff_fig = plt.figure(7)
#     plt.plot(day, sdjfq_ff[0], 'b', label="CPU Usage")
#     plt.plot(day, sdjfq_ff[1], 'g', label="Mem Usage")
#     plt.legend()
#     plt.title('Shortest Duration Job First - First Fit Usage')
# # ---------------------------------------------------------------------------------------------------------------------
#     # 8th  Combination
#     print("\nShortest Duration Job First with Best Fit")
#     sdjfq_bf = ms_node.find_usage("SDJFQ", "BF")

#     print("CPU Usage :", sdjfq_bf[0], "\nMem Usage :", sdjfq_bf[1])

#     sdjfq_bf = give_percentage(sdjfq_bf[0], sdjfq_bf[1])
#     # print("CPU Usage in %:", fcfs_ff[0], "\nMem Usage in %:", fcfs_ff[1])

#     sdjfq_bf_fig = plt.figure(8)
#     plt.plot(day, sdjfq_bf[0], 'b', label="CPU Usage")
#     plt.plot(day, sdjfq_bf[1], 'g', label="Mem Usage")
#     plt.legend()
#     plt.title('Shortest Duration Job First - Best Fit Usage')
# # ---------------------------------------------------------------------------------------------------------------------
#     # 9th  Combination
#     print("\nShortest Duration Job First with Worst Fit")
#     sdjfq_wf = ms_node.find_usage("SDJFQ", "WF")

#     print("CPU Usage :", sdjfq_wf[0], "\nMem Usage :", sdjfq_wf[1])

#     sdjfq_wf = give_percentage(sdjfq_wf[0], sdjfq_wf[1])
#     # print("CPU Usage in %:", fcfs_ff[0], "\nMem Usage in %:", fcfs_ff[1])

#     sdjfq_wf_fig = plt.figure(9)
#     plt.plot(day, sdjfq_wf[0], 'b', label="CPU Usage")
#     plt.plot(day, sdjfq_wf[1], 'g', label="Mem Usage")
#     plt.legend()
#     plt.title('Shortest Duration Job First - Worst Fit Usage')

#     plt.show()

except Exception as e:
    print("Some Error Occurred")
    print(e)


FCFS with First Fit
([1547.4166666666667, 2033.625, 1573.7083333333333, 2402.9583333333335, 1704.375, 2010.7916666666667, 2124.2916666666665, 2063.4166666666665, 1988.25, 2028.5416666666667, 1713.1666666666667, 1716.0416666666667, 1860.4583333333333, 1952.4166666666667, 1537.125, 1808.7083333333333, 2073.6666666666665, 1792.75, 1675.5, 1993.5, 1875.875, 1928.4166666666667, 2116.75, 1924.7083333333333, 1653.5, 1878.4583333333333, 2083.9583333333335, 1686.9583333333333, 2063.9166666666665, 2410.6666666666665], [2957.2083333333335, 3839.4166666666665, 3023.125, 4591.666666666667, 3079.75, 3795.5416666666665, 3944.75, 3865.75, 3759.75, 3664.8333333333335, 3335.4583333333335, 3142.7916666666665, 3589.7916666666665, 3787.375, 2919.8333333333335, 3576.4583333333335, 4024.4166666666665, 3439.2083333333335, 3170.25, 3765.0416666666665, 3648.5416666666665, 3809.875, 4116.041666666667, 3698.5416666666665, 3042.125, 3622.0416666666665, 4013.0833333333335, 3278.125, 3871.0833333333335, 4631.541666

# Threading

## Without threading Timesort

In [20]:
import random
import time

for length in [10,100,1000,10000,100000, 1000000, 10000000]:
    average_time = 0
    for _ in range(10):
        A = [random.randint(1,100000) for i in range(length)]
        start_time = time.time()
        A = sorted(A)
        average_time += (time.time()-start_time)
    print("Length {0} executed in {1} seconds".format(length, average_time/10))

Length 10 executed in 2.002716064453125e-06 seconds
Length 100 executed in 2.2149085998535158e-05 seconds
Length 1000 executed in 0.00022511482238769532 seconds
Length 10000 executed in 0.001522994041442871 seconds
Length 100000 executed in 0.020492815971374513 seconds
Length 1000000 executed in 0.30686795711517334 seconds
Length 10000000 executed in 4.5422461986541744 seconds


## Without threading

In [9]:
def mergeSort(arr):
	if len(arr) > 1:

		mid = len(arr)//2

		L = arr[:mid]

		R = arr[mid:]

		mergeSort(L)

		mergeSort(R)

		i = j = k = 0

		while i < len(L) and j < len(R):
			if L[i] < R[j]:
				arr[k] = L[i]
				i += 1
			else:
				arr[k] = R[j]
				j += 1
			k += 1

		while i < len(L):
			arr[k] = L[i]
			i += 1
			k += 1

		while j < len(R):
			arr[k] = R[j]
			j += 1
			k += 1
	return arr

for length in [10,100,1000,10000,100000, 1000000, 10000000]:
    average_time = 0
    for _ in range(3):
        A = [random.randint(1,100000) for i in range(length)]
        start_time = time.time()
        A = mergeSort(A)
        average_time += (time.time()-start_time)
    print("Length {0} executed in {1} seconds".format(length, average_time/3))

Length 10 executed in 5.332628885904948e-05 seconds
Length 100 executed in 0.0005736351013183594 seconds
Length 1000 executed in 0.005640347798665364 seconds
Length 10000 executed in 0.05061729749043783 seconds
Length 100000 executed in 0.6185946464538574 seconds
Length 1000000 executed in 8.00765577952067 seconds
Length 10000000 executed in 111.48988493283589 seconds


## Multithreading with shared data(globally defined data)

In [10]:
import threading

has_threaded = True

def mergeSort(arr):
    
#     print("Called mergesort",threading.current_thread().name, arr[0],arr[-1])
    
    if len(arr) > 1:

        mid = len(arr)//2

        L = arr[:mid]

        R = arr[mid:]

        global has_threaded
        
        if has_threaded:
#             print("Threaded")
            has_threaded = False
            t1 = threading.Thread(target = mergeSort, args=(L,),name='left')
            t2 = threading.Thread(target = mergeSort, args=(R,),name='right')
            t1.start()
            t2.start()
            
            t1.join()
            t2.join()
            
        else:
            mergeSort(L)
            mergeSort(R)
            
        i = j = k = 0

        while i < len(L) and j < len(R):
            if L[i] < R[j]:
                arr[k] = L[i]
                i += 1
            else:
                arr[k] = R[j]
                j += 1
            k += 1

        while i < len(L):
            arr[k] = L[i]
            i += 1
            k += 1

        while j < len(R):
            arr[k] = R[j]
            j += 1
            k += 1
    return arr

for length in [10,100,1000,10000,100000, 1000000, 10000000]:
    average_time = 0
    for _ in range(3):
        A = [random.randint(1,100000) for i in range(length)]
        start_time = time.time()
        A = mergeSort(A)
        average_time += (time.time()-start_time)
    print("Length {0} executed in {1} seconds".format(length, average_time/3))

Length 10 executed in 0.00025312105814615887 seconds
Length 100 executed in 0.00032671292622884113 seconds
Length 1000 executed in 0.0048720041910807295 seconds
Length 10000 executed in 0.05094639460245768 seconds
Length 100000 executed in 0.6209287643432617 seconds
Length 1000000 executed in 7.80043641726176 seconds
Length 10000000 executed in 104.67939352989197 seconds


## Multithreading with unshared data

In [8]:
import copy
import threading
import random

has_threaded = True

def mergeSort(arr):
    
#     print("Called mergesort",threading.current_thread().name, arr[0],arr[-1])
    
    if len(arr) > 1:

        mid = len(arr)//2
        

        L = arr[:mid]

        R = arr[mid:]

        mergeSort(L)
        mergeSort(R)

        i = j = k = 0

        while i < len(L) and j < len(R):
            if L[i] < R[j]:
                arr[k] = L[i]
                i += 1
            else:
                arr[k] = R[j]
                j += 1
            k += 1

        while i < len(L):
            arr[k] = L[i]
            i += 1
            k += 1

        while j < len(R):
            arr[k] = R[j]
            j += 1
            k += 1
    return arr

def merge(A,B):
    i = j = k = 0
#         print("joining")
#         t = time.time()
    N, M = len(A), len(B)
    arr = [0]*(N+M)
    while i < N and j < M:
        if A[i] < B[j]:
            arr[k] = A[i]
            i += 1
        else:
            arr[k] = B[j]
            j += 1
        k += 1

    while i < N:
        arr[k] = A[i]
        i += 1
        k += 1

    while j < M:
        arr[k] = B[j]
        j += 1
        k += 1
    
    return arr

for length in [10,100,1000,10000,100000, 1000000, 10000000]:
    average_time = 0
    for _ in range(3):

        A = [random.randint(1,100000) for i in range(length//3)]
        B = [random.randint(1,100000) for i in range(length//3)]
        D = [random.randint(1,100000) for i in range(length//3)]
        
        t1 = threading.Thread(target = mergeSort, args=(A,),name=str('left'))
        t2 = threading.Thread(target = mergeSort, args=(B,),name=str('right'))
        t3 = threading.Thread(target = mergeSort, args=(D,),name=str('center'))
        
        start_time = time.time()
        
        t1.start()
        t2.start()
        t3.start()
            
        t1.join()
        t2.join()
        t3.join()

        C = merge(A,B)
#         print(len(C))
        ans = merge(C,D)
        
#         print("joining2")
#         print(time.time()-t,length,_)
        average_time += (time.time()-start_time)
    print("Length {0} executed in {1} seconds".format(length, average_time/3))

Length 10 executed in 0.0010461807250976562 seconds
Length 100 executed in 0.001554568608601888 seconds
Length 1000 executed in 0.006805737813313802 seconds
Length 10000 executed in 0.04734206199645996 seconds
Length 100000 executed in 0.6109469731648763 seconds
Length 1000000 executed in 7.697932243347168 seconds
Length 10000000 executed in 112.1619819800059 seconds


## Using Multi-Processing 

In [20]:
import copy
import multiprocessing
import random

has_threaded = True

def mergeSort(arr):
    
#     print("Called mergesort",threading.current_thread().name, arr[0],arr[-1])
    
    if len(arr) > 1:

        mid = len(arr)//2
        

        L = arr[:mid]

        R = arr[mid:]

        mergeSort(L)
        mergeSort(R)

        i = j = k = 0

        while i < len(L) and j < len(R):
            if L[i] < R[j]:
                arr[k] = L[i]
                i += 1
            else:
                arr[k] = R[j]
                j += 1
            k += 1

        while i < len(L):
            arr[k] = L[i]
            i += 1
            k += 1

        while j < len(R):
            arr[k] = R[j]
            j += 1
            k += 1
    return arr

def merge(A,B):
    i = j = k = 0
#         print("joining")
#         t = time.time()
    N, M = len(A), len(B)
    arr = [0]*(N+M)
    while i < N and j < M:
        if A[i] < B[j]:
            arr[k] = A[i]
            i += 1
        else:
            arr[k] = B[j]
            j += 1
        k += 1

    while i < N:
        arr[k] = A[i]
        i += 1
        k += 1

    while j < M:
        arr[k] = B[j]
        j += 1
        k += 1
    
    return arr

for length in [10,100,1000,10000,100000, 1000000, 10000000]:
    average_time = 0
    for _ in range(3):

        A = [random.randint(1,100000) for i in range(length//3)]
        B = [random.randint(1,100000) for i in range(length//3)]
        D = [random.randint(1,100000) for i in range(length//3)]
        
        t1 = multiprocessing.Process(target = mergeSort, args=(A,),name=str('left'))
        t2 = multiprocessing.Process(target = mergeSort, args=(B,),name=str('right'))
        t3 = multiprocessing.Process(target = mergeSort, args=(D,),name=str('center'))
        
        start_time = time.time()
        
        t1.start()
        t2.start()
        t3.start()
            
        t1.join()
        t2.join()
        t3.join()

        C = merge(A,B)
#         print(len(C))
        ans = merge(C,D)
        
#         print("joining2")
#         print(time.time()-t,length,_)
        average_time += (time.time()-start_time)
    print("Length {0} executed in {1} seconds".format(length, average_time/3))

Length 10 executed in 0.15581115086873373 seconds
Length 100 executed in 0.11199617385864258 seconds
Length 1000 executed in 0.11774706840515137 seconds
Length 10000 executed in 0.1278203328450521 seconds
Length 100000 executed in 0.45400500297546387 seconds
Length 1000000 executed in 4.050913651784261 seconds
Length 10000000 executed in 48.647414684295654 seconds


In [18]:
length = 10000000

A = [random.randint(1,100000) for i in range(length//10)]
B = [random.randint(1,100000) for i in range(length//10)]
C = [random.randint(1,100000) for i in range(length//10)]
D = [random.randint(1,100000) for i in range(length//10)]
E = [random.randint(1,100000) for i in range(length//10)]
F = [random.randint(1,100000) for i in range(length//10)]
G = [random.randint(1,100000) for i in range(length//10)]
H = [random.randint(1,100000) for i in range(length//10)]
I = [random.randint(1,100000) for i in range(length//10)]
J = [random.randint(1,100000) for i in range(length//10)]
K = [random.randint(1,100000) for i in range(length//10)]

t1 = threading.Thread(target = mergeSort, args=(A,),name=str('1'))
t2 = threading.Thread(target = mergeSort, args=(B,),name=str('2'))
t3 = threading.Thread(target = mergeSort, args=(C,),name=str('3'))
t4 = threading.Thread(target = mergeSort, args=(D,),name=str('4'))
t5 = threading.Thread(target = mergeSort, args=(E,),name=str('5'))
t6 = threading.Thread(target = mergeSort, args=(F,),name=str('6'))
t7 = threading.Thread(target = mergeSort, args=(G,),name=str('7'))
t8 = threading.Thread(target = mergeSort, args=(H,),name=str('8'))
t9 = threading.Thread(target = mergeSort, args=(I,),name=str('9'))
t10 = threading.Thread(target = mergeSort, args=(J,),name=str('10'))
t11 = threading.Thread(target = mergeSort, args=(K,),name=str('11'))

start_time = time.time()

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()
t7.start()
t8.start()
t9.start()
t10.start()
t11.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()
t6.join()
t7.join()
t8.join()
t9.join()
t10.join()
t11.join()

# ans = merge(A,B)
# ans = merge(ans,C)
# ans = merge(ans,D)
# ans = merge(ans,E)
# ans = merge(ans,F)
# ans = merge(ans,G)
# ans = merge(ans,H)
# ans = merge(ans,I)
# ans = merge(ans,J)
# ans = merge(ans,K)

merge_start_time = time.time()

ans0 = merge(A,B)
ans1 = merge(C,D)
ans2 = merge(E,F)
ans3 = merge(G,H)
ans4 = merge(I,J)

ans5 = merge(K,ans0)
ans6 = merge(ans1,ans2)
ans7 = merge(ans3,ans4)

ans8 = merge(ans5,ans6)
ans = merge(ans8,ans7)

print(time.time()-start_time, time.time()-merge_start_time)

108.04576778411865 14.188493013381958


In [19]:
import multiprocessing

length = 10000000

A = [random.randint(1,100000) for i in range(length//10)]
B = [random.randint(1,100000) for i in range(length//10)]
C = [random.randint(1,100000) for i in range(length//10)]
D = [random.randint(1,100000) for i in range(length//10)]
E = [random.randint(1,100000) for i in range(length//10)]
F = [random.randint(1,100000) for i in range(length//10)]
G = [random.randint(1,100000) for i in range(length//10)]
H = [random.randint(1,100000) for i in range(length//10)]
I = [random.randint(1,100000) for i in range(length//10)]
J = [random.randint(1,100000) for i in range(length//10)]
K = [random.randint(1,100000) for i in range(length//10)]

t1 = multiprocessing.Process(target = mergeSort, args=(A,),name=str('1'))
t2 = multiprocessing.Process(target = mergeSort, args=(B,),name=str('2'))
t3 = multiprocessing.Process(target = mergeSort, args=(C,),name=str('3'))
t4 = multiprocessing.Process(target = mergeSort, args=(D,),name=str('4'))
t5 = multiprocessing.Process(target = mergeSort, args=(E,),name=str('5'))
t6 = multiprocessing.Process(target = mergeSort, args=(F,),name=str('6'))
t7 = multiprocessing.Process(target = mergeSort, args=(G,),name=str('7'))
t8 = multiprocessing.Process(target = mergeSort, args=(H,),name=str('8'))
t9 = multiprocessing.Process(target = mergeSort, args=(I,),name=str('9'))
t10 = multiprocessing.Process(target = mergeSort, args=(J,),name=str('10'))
t11 = multiprocessing.Process(target = mergeSort, args=(K,),name=str('11'))

start_time = time.time()

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()
t7.start()
t8.start()
t9.start()
t10.start()
t11.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()
t6.join()
t7.join()
t8.join()
t9.join()
t10.join()
t11.join()

# ans = merge(A,B)
# ans = merge(ans,C)
# ans = merge(ans,D)
# ans = merge(ans,E)
# ans = merge(ans,F)
# ans = merge(ans,G)
# ans = merge(ans,H)
# ans = merge(ans,I)
# ans = merge(ans,J)
# ans = merge(ans,K)

merge_start_time = time.time()

ans0 = merge(A,B)
ans1 = merge(C,D)
ans2 = merge(E,F)
ans3 = merge(G,H)
ans4 = merge(I,J)

ans5 = merge(K,ans0)
ans6 = merge(ans1,ans2)
ans7 = merge(ans3,ans4)

ans8 = merge(ans5,ans6)
ans = merge(ans8,ans7)

print(time.time()-start_time, time.time()-merge_start_time)

51.97875261306763 7.929607629776001


In [ ]:
# Python3 program to perform basic timSort 
MIN_MERGE = 3

def min(a,b):
    if a>b:
        return b
    return a

def calcMinRun(n): 
    r = 0
    while n >= MIN_MERGE: 
        r |= n & 1
        n >>= 1
    return n + r 

def insertionSort(arr, left, right): 
    for i in range(left + 1, right + 1): 
        j = i 
        while j > left and arr[j] < arr[j - 1]: 
            arr[j], arr[j - 1] = arr[j - 1], arr[j] 
            j -= 1

def merge(arr, l, m, r): 
 
    len1, len2 = m - l + 1, r - m 
    left, right = [], [] 
    for i in range(0, len1): 
        left.append(arr[l + i]) 
    for i in range(0, len2): 
        right.append(arr[m + 1 + i]) 

    i, j, k = 0, 0, l 

    while i < len1 and j < len2: 
        if left[i] <= right[j]: 
            arr[k] = left[i] 
            i += 1

        else: 
            arr[k] = right[j] 
            j += 1

        k += 1

    while i < len1: 
        arr[k] = left[i] 
        k += 1
        i += 1

    while j < len2: 
        arr[k] = right[j] 
        k += 1
        j += 1

def timSort(arr): 
    
    n = len(arr) 
    minRun = calcMinRun(n) 
    
    process = []
    
    for start in range(0, n, minRun): 
        end = min(start + minRun - 1, n - 1) 
        process.append(multiprocessing.Process(target = insertionSort, args = (arr, start, end) ))
        
    for p in process:
        p.start()

    for p in process:
        p.join()
        
    size = minRun 
    while size < n: 

        for left in range(0, n, 2 * size): 

            mid = min(n - 1, left + size - 1) 
            right = min((left + 2 * size - 1), (n - 1)) 
            merge(arr, left, mid, right) 

        size = 2 * size 

if __name__ == "__main__": 

    arr = [random.randint(1,10000)]*10000000
    
    s = time.time()

    timSort(arr) 
    
    print(time.time()-s)


Exception ignored in: <function _releaseLock at 0x7faf72ad3b00>
Traceback (most recent call last):
  File "/home/sanket/anaconda3/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Exception ignored in: <function _releaseLock at 0x7faf72ad3b00>
Traceback (most recent call last):
  File "/home/sanket/anaconda3/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Exception ignored in: <function _releaseLock at 0x7faf72ad3b00>
Traceback (most recent call last):
  File "/home/sanket/anaconda3/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Exception ignored in: <function _releaseLock at 0x7faf72ad3b00>
Traceback (most recent call last):
  File "/home/sanket/anaconda3/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Exception ignored in: <function _releaseLock at 0x7faf72ad3b00>


## Multiprocessing TimeSort

In [ ]:
import copy
import multiprocessing
import random

has_threaded = True

def mergeSort(arr):
    
#     print("Called mergesort",threading.current_thread().name, arr[0],arr[-1])
    
    if len(arr) > 1:

        mid = len(arr)//2
        

        L = arr[:mid]

        R = arr[mid:]

        mergeSort(L)
        mergeSort(R)

        i = j = k = 0

        while i < len(L) and j < len(R):
            if L[i] < R[j]:
                arr[k] = L[i]
                i += 1
            else:
                arr[k] = R[j]
                j += 1
            k += 1

        while i < len(L):
            arr[k] = L[i]
            i += 1
            k += 1

        while j < len(R):
            arr[k] = R[j]
            j += 1
            k += 1
    return arr

def merge(A,B):
    i = j = k = 0
#         print("joining")
#         t = time.time()
    N, M = len(A), len(B)
    arr = [0]*(N+M)
    while i < N and j < M:
        if A[i] < B[j]:
            arr[k] = A[i]
            i += 1
        else:
            arr[k] = B[j]
            j += 1
        k += 1

    while i < N:
        arr[k] = A[i]
        i += 1
        k += 1

    while j < M:
        arr[k] = B[j]
        j += 1
        k += 1
    
    return arr

for length in [10,100,1000,10000,100000, 1000000, 10000000]:
    average_time = 0
    for _ in range(3):

        A = [random.randint(1,100000) for i in range(length//3)]
        B = [random.randint(1,100000) for i in range(length//3)]
        D = [random.randint(1,100000) for i in range(length//3)]
        
        t1 = multiprocessing.Process(target = mergeSort, args=(A,),name=str('left'))
        t2 = multiprocessing.Process(target = mergeSort, args=(B,),name=str('right'))
        t3 = multiprocessing.Process(target = mergeSort, args=(D,),name=str('center'))
        
        start_time = time.time()
        
        t1.start()
        t2.start()
        t3.start()
            
        t1.join()
        t2.join()
        t3.join()

        C = merge(A,B)
#         print(len(C))
        ans = merge(C,D)
        
#         print("joining2")
#         print(time.time()-t,length,_)
        average_time += (time.time()-start_time)
    print("Length {0} executed in {1} seconds".format(length, average_time/3))

# Sorting Algorithm comparision

In [32]:
import random
import time

data = []
data.append([random.randint(1,10) for _ in range(1000000)])
data.append([random.randint(1,2) for _ in range(1000000)])
data.append(list(range(1000000)))
a = list(range(1000000))
random.shuffle(a)
data.append(a)
data.append(list(range(500000))+list(range(500000)))

print(len(data))

# TimeSort
for d in data:
    start_time = time.time()
    a = sorted(d)
    print(time.time()-start_time)

# MergeSort
for d in data:
    start_time = time.time()
    a = MergeSort(d)
    print(time.time()-start_time)
    
# QuickSort
for d in data:
    start_time = time.time()
    a = QuickSort(d)
    print(time.time()-start_time)
    
# InsertSort
for d in data:
    start_time = time.time()
    a = insertSort(d)
    print(time.time()-start_time)
    
# HeapSort
for d in data:
    start_time = time.time()
    a = HeapSort(d)
    print(time.time()-start_time)    
    
del data

5
0.09264659881591797
0.05205273628234863
0.014255523681640625
0.45501255989074707
0.023838043212890625
